In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K

# List of companies to fetch data for
companies = [
    "ASIANPAINT", "ASTRAL", "BHARATFORG", "BHARTIARTL", "CIPLA",
    "DEEPAKNTR", "DIXON", "EXIDEIND", "GRASIM", "HAL",
    "INDUSINDBK", "INFY", "OBEROIRLTY", "RELIANCE", "SBIN",
    "TATAMOTORS", "TCS", "TRENT", "ZEEL", "ULTRACEMCO"
]

# Fetch data for each company and store in a dictionary
company_data = {}
for company in companies:
    symbol = company + ".NS"  # Append .NS for NSE symbols in yfinance
    data = yf.download(symbol, start="2023-01-01", end="2023-10-01")
    if data.empty:
        print(f"No data found for {company}")
    else:
        company_data[company] = data['Adj Close']
        print(f"Data for {company} fetched successfully!")

# Concatenate data for all companies and calculate daily returns
price_data = pd.concat(company_data.values(), axis=1)
price_data.columns = companies
returns = price_data.pct_change().dropna()

# Normalize returns
scaler = MinMaxScaler()
returns_scaled = scaler.fit_transform(returns)

# Prepare data for LSTM
window_size = 30
X, y = [], []

for i in range(window_size, len(returns_scaled)):
    X.append(returns_scaled[i-window_size:i])
    y.append(returns_scaled[i])

X, y = np.array(X), np.array(y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define custom loss function to maximize return, minimize risk, and maximize entropy
def portfolio_loss(y_true, y_pred):
    # Normalize predicted weights
    weights = K.softmax(y_pred)

    # Calculate portfolio return (maximize return)
    portfolio_return = K.sum(weights * y_true, axis=1)

    # Calculate portfolio risk (minimize risk)
    portfolio_risk = K.sqrt(K.sum(K.square(y_true - K.mean(y_true, axis=0)) * weights, axis=1))

    # Calculate portfolio entropy (maximize entropy)
    portfolio_entropy = -K.sum(weights * K.log(weights + 1e-9), axis=1)

    # Final loss: adjust weights to trade off return, risk, and entropy
    loss = -K.mean(portfolio_return) + K.mean(portfolio_risk) - K.mean(portfolio_entropy)

    return loss

# Build the LSTM model to predict portfolio weights
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(window_size, len(companies))))
model.add(Dense(len(companies)))  # Output shape will be (None, num_companies)
model.compile(optimizer='adam', loss=portfolio_loss)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions (optimal weights)
predicted_weights = model.predict(X_test)

# Apply softmax to ensure weights sum to 1
predicted_weights = np.exp(predicted_weights) / np.sum(np.exp(predicted_weights), axis=1, keepdims=True)

# Calculate portfolio metrics using predicted weights
for i, weights in enumerate(predicted_weights[:5]):
    selected_companies = companies[:len(weights)]  # Match companies to predicted weights

    mean_returns = returns[selected_companies].mean()
    portfolio_expected_return = np.dot(weights, mean_returns)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(returns[selected_companies].cov(), weights)))
    portfolio_entropy = -np.sum(weights * np.log(weights + 1e-9))

    print(f"Portfolio {i+1}:")
    print(f"  Weights: {weights}")
    print(f"  Expected Return: {portfolio_expected_return}")
    print(f"  Risk (Standard Deviation): {portfolio_risk}")
    print(f"  Entropy: {portfolio_entropy}\n")


[*********************100%***********************]  1 of 1 completed


Data for ASIANPAINT fetched successfully!


[*********************100%***********************]  1 of 1 completed


Data for ASTRAL fetched successfully!


[*********************100%***********************]  1 of 1 completed


...

Data for TCS fetched successfully!
Data for TRENT fetched successfully!


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Data for ZEEL fetched successfully!
Data for ULTRACEMCO fetched successfully!


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 169ms/step - loss: -3.3291 - val_loss: -3.3686
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: -3.3457 - val_loss: -3.3730
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: -3.3450 - val_loss: -3.3745
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: -3.3459 - val_loss: -3.3756
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: -3.3539 - val_loss: -3.3765
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: -3.3579 - val_loss: -3.3771
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: -3.3515 - val_loss: -3.3774
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: -3.3533 - val_loss: -3.3776
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: -3.3570 - val_loss: -3.3776
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: -3.3542 - val_loss: -3.3776
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: -3.3529 - val_loss: -3.3776
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: -3.3